In [175]:
# Importamos librerías necesarias
import telebot
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [ ]:
# Variables de configuración
TOKEN = ''
CSV_FILE = '..\data\processed\Articulos_LLM6.csv'  # ← tu archivo CSV

# Cargamos el CSV de artículos
articulos_df = pd.read_csv(CSV_FILE)
articulos_df = articulos_df.drop_duplicates()

# Vamos a unir titulo + contenido + fechas + precios + edad
articulos_df['texto_completo'] = (
    articulos_df['titulo'].fillna('') + '. ' +
    articulos_df['contenido'].fillna('') + ' ' +
    articulos_df['fechas_contexto'].fillna('') + ' ' +
    articulos_df['Precios'].fillna('') + ' '
)

In [177]:
import nltk
from nltk.corpus import stopwords

# Asegúrate de tener descargado el paquete de stopwords de NLTK
nltk.download('stopwords')

# Cargamos las stopwords en español
stopwords_es = stopwords.words('spanish')

# Creamos el TF-IDF usando las stopwords en español
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_es)

# Ahora sí, transformamos
tfidf_matrix = tfidf_vectorizer.fit_transform(articulos_df['texto_completo'])


[nltk_data] Downloading package stopwords to C:\Users\Abdon.RAMIREZ-
[nltk_data]     BRICEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [179]:
from datetime import datetime, timedelta, date
import dateparser
import re
import telebot
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Inicializamos el bot
bot = telebot.TeleBot(TOKEN)

# --- Interpretar fechas desde la consulta --

# Interpretar fechas desde pregunta del usuario
def interpretar_fecha_usuario(pregunta):
    pregunta = pregunta.lower()
    hoy = datetime.now()

    if 'hoy' in pregunta:
        return hoy.date(), hoy.date()
    elif 'mañana' in pregunta:
        manana = hoy + timedelta(days=1)
        return manana.date(), manana.date()
    elif 'semana que viene' in pregunta or 'semana próxima' in pregunta or 'semana siguiente' in pregunta:
        inicio = hoy + timedelta(days=(7 - hoy.weekday()))
        fin = inicio + timedelta(days=6)
        return inicio.date(), fin.date()
    elif 'esta semana' in pregunta:
        inicio = hoy - timedelta(days=hoy.weekday())
        fin = inicio + timedelta(days=6)
        return inicio.date(), fin.date()
    elif 'fin de semana' in pregunta or 'sábado' in pregunta or 'domingo' in pregunta:
        dias_hasta_sabado = (5 - hoy.weekday()) % 7
        sabado = hoy + timedelta(days=dias_hasta_sabado)
        domingo = sabado + timedelta(days=1)
        if hoy.weekday() == 6:  # domingo
            return hoy.date(), hoy.date()
        return sabado.date(), domingo.date()
    else:
        return None, None

# Detectar ciudad
def detectar_ciudad(pregunta):
    ciudades = ["madrid", "barcelona", "malaga", "sevilla", "valencia", "zaragoza"]
    pregunta = pregunta.lower()
    for ciudad in ciudades:
        if ciudad in pregunta:
            return ciudad.capitalize()
    return None

# Preparar fechas dataset
def preparar_fechas_dataset(df):
    fecha_inicio_parseada = []
    fecha_fin_parseada = []

    for inicio, fin in zip(df['fecha_inicio'].fillna(''), df['fecha_fin'].fillna('')):
        inicio_dt = dateparser.parse(str(inicio), languages=['es'])
        fecha_inicio_parseada.append(inicio_dt.date() if inicio_dt else None)

        fin_dt = dateparser.parse(str(fin), languages=['es'])
        fecha_fin_parseada.append(fin_dt.date() if fin_dt else None)

    df['fecha_inicio_parseada'] = fecha_inicio_parseada
    df['fecha_fin_parseada'] = fecha_fin_parseada

    df['fecha_fin_parseada'] = pd.to_datetime(df['fecha_fin_parseada'], errors='coerce')
    df['fecha_inicio_parseada'] = pd.to_datetime(df['fecha_inicio_parseada'], errors='coerce')
    return df

def filtrar_eventos_futuros(df):
    hoy = pd.Timestamp(datetime.now().date())

    filtrados = df[
        (df['fecha_fin_parseada'].notnull()) & (df['fecha_fin_parseada'] >= hoy) &
        (df['fecha_inicio_parseada'].notnull()) & (df['fecha_inicio_parseada'] >= hoy)
    ]

    return filtrados


# Suponiendo que ya tienes articulos_df y tfidf_vectorizer cargados
articulos_df = preparar_fechas_dataset(articulos_df)
articulos_df = filtrar_eventos_futuros(articulos_df)

# --- Buscar y responder actividad ---
def buscar_respuesta(pregunta_usuario):
    hoy = pd.Timestamp(datetime.now().date())
    fecha_inicio_consulta, fecha_fin_consulta = interpretar_fecha_usuario(pregunta_usuario)
    ciudad_consulta = detectar_ciudad(pregunta_usuario)

    print(f"DEBUG 🛠 Fecha: {fecha_inicio_consulta} a {fecha_fin_consulta}")
    print(f"DEBUG 🛠 Ciudad: {ciudad_consulta}")

    filtrados = articulos_df.copy()
    # Ya no necesitas hacer esto si lo hiciste en preparar_fechas_dataset:
    # articulos_df['fecha_fin_parseada'] = pd.to_datetime(...)
    # Filtro robusto y eficiente sin apply
    filtrados = filtrados[
        (filtrados['fecha_fin_parseada'].notnull()) & (filtrados['fecha_fin_parseada'] >= hoy) &
        (filtrados['fecha_inicio_parseada'].notnull()) & (filtrados['fecha_inicio_parseada'] >= hoy)
    ]

    
    if fecha_inicio_consulta and fecha_fin_consulta:
        fecha_inicio_consulta_ts = pd.Timestamp(fecha_inicio_consulta)
        fecha_fin_consulta_ts = pd.Timestamp(fecha_fin_consulta)

        filtrados = filtrados[
            (filtrados['fecha_inicio_parseada'].apply(lambda x: isinstance(x, date) and x <= fecha_fin_consulta_ts)) &
            (filtrados['fecha_fin_parseada'].apply(lambda x: isinstance(x, date) and x >= fecha_inicio_consulta_ts))
        ]

    if ciudad_consulta:
        filtrados = filtrados[filtrados['texto_completo'].str.lower().str.contains(ciudad_consulta.lower())]

    if filtrados.empty:
        return "🤔 No encontré actividades para esa ciudad y fecha. ¡Intenta preguntar otra cosa!"

    # TF-IDF búsqueda
    tfidf_matrix_filtrada = tfidf_vectorizer.transform(filtrados['texto_completo'])
    pregunta_vectorizada = tfidf_vectorizer.transform([pregunta_usuario])
    similitudes = cosine_similarity(pregunta_vectorizada, tfidf_matrix_filtrada)

    filtrados = filtrados.copy()
    filtrados['similitud'] = similitudes[0]
    filtrados = filtrados.sort_values(by='similitud', ascending=False)

    # Evitar repetir URLs en la misma respuesta
    urls_vistas = set()
    respuesta = ""
    count = 0

    for _, articulo in filtrados.iterrows():
        if articulo['url'] in urls_vistas:
            continue
        urls_vistas.add(articulo['url'])

        respuesta += (
            f"✅ *{articulo['titulo']}*\n"
            f"{articulo['contenido'][:300]}...\n"
            f"💰 Precio: {articulo['Precios']}\n"
            f"🔗 [Más info]({articulo['url']})\n\n"
        )

        count += 1
        if count == 3:
            break

    return respuesta if respuesta else "🤔 No encontré actividades únicas para mostrar."

# --- Bot handlers ---
@bot.message_handler(commands=['start'])
def start(message):
    chat_id = message.chat.id
    with open('..\QHCN.jpg', 'rb') as photo:
        bot.send_photo(chat_id, photo)

    mensaje_bienvenida = (
        "👋 ¡Hola! Soy tu asistente virtual para ayudarte a buscar actividades.\n\n"
        "Puedes preguntarme cosas como:\n"
        "📌 ¿Qué actividad puedo hacer hoy?\n"
        "📌 A mi hijo/a le gusta el deporte, ¿qué podría hacer esta semana?\n"
        "📌 ¿Qué actividades hay en Sevilla hoy?\n"
        "📌 ¿Qué actividades puedo hacer este fin de semana gratis?\n\n"
        "¡Estoy aquí para ayudarte! 🎯"
    )
    bot.send_message(chat_id, mensaje_bienvenida, parse_mode='Markdown')

@bot.message_handler(func=lambda message: True)
def responder(message):
    consulta = message.text
    respuesta = buscar_respuesta(consulta)
    bot.send_message(message.chat.id, respuesta, parse_mode='Markdown')

# --- Arranque del bot ---
print("🤖 Bot activo y esperando mensajes...")
bot.polling()


🤖 Bot activo y esperando mensajes...
DEBUG 🛠 Fecha: None a None
DEBUG 🛠 Ciudad: None


In [14]:
df  = preparar_fechas_dataset(articulos_df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ciudad                 690 non-null    object
 1   seccion                690 non-null    object
 2   titulo                 690 non-null    object
 3   url                    690 non-null    object
 4   contenido              690 non-null    object
 5   fecha_inicio           372 non-null    object
 6   fecha_fin              372 non-null    object
 7   fechas                 690 non-null    object
 8   Precios                690 non-null    object
 9   fechas_contexto        366 non-null    object
 10  texto_completo         690 non-null    object
 11  fecha_inicio_parseada  372 non-null    object
 12  fecha_fin_parseada     372 non-null    object
dtypes: object(13)
memory usage: 70.2+ KB


In [ ]:
df['fecha_fin_parseada'] = pd.to_datetime(df['fecha_fin_parseada'], errors='coerce')
df['fecha_inicio_parseada'] = pd.to_datetime(df['fecha_inicio_parseada'], errors='coerce')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ciudad                 690 non-null    object        
 1   seccion                690 non-null    object        
 2   titulo                 690 non-null    object        
 3   url                    690 non-null    object        
 4   contenido              690 non-null    object        
 5   fecha_inicio           372 non-null    object        
 6   fecha_fin              372 non-null    object        
 7   fechas                 690 non-null    object        
 8   Precios                690 non-null    object        
 9   fechas_contexto        366 non-null    object        
 10  texto_completo         690 non-null    object        
 11  fecha_inicio_parseada  371 non-null    datetime64[ns]
 12  fecha_fin_parseada     370 non-null    datetime64[ns]
dtypes: da

In [ ]:
hoy = datetime.now()
filtrados = df[df['fecha_fin_parseada'].apply(lambda x: isinstance(x, date) and x >= hoy)]
filtrados = filtrados[filtrados['fecha_inicio_parseada'].apply(lambda x: isinstance(x, date) and x >= hoy)]

In [ ]:
filtrados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 3 to 676
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ciudad                 185 non-null    object        
 1   seccion                185 non-null    object        
 2   titulo                 185 non-null    object        
 3   url                    185 non-null    object        
 4   contenido              185 non-null    object        
 5   fecha_inicio           185 non-null    object        
 6   fecha_fin              185 non-null    object        
 7   fechas                 185 non-null    object        
 8   Precios                185 non-null    object        
 9   fechas_contexto        183 non-null    object        
 10  texto_completo         185 non-null    object        
 11  fecha_inicio_parseada  185 non-null    datetime64[ns]
 12  fecha_fin_parseada     185 non-null    datetime64[ns]
dtypes: datetim

In [ ]:
fecha_inicio_consulta, fecha_fin_consulta = interpretar_fecha_usuario('que actividad hay para madrid hoy')
ciudad_consulta = detectar_ciudad('que actividad hay para madrid hoy')

In [ ]:
# Al preparar el dataset, asegúrate de hacer esto:
df['fecha_inicio_parseada'] = pd.to_datetime(df['fecha_inicio'], errors='coerce')
df['fecha_fin_parseada'] = pd.to_datetime(df['fecha_fin'], errors='coerce')

# Y conviertes las fechas de consulta a Timestamp:
fecha_inicio_consulta_ts = pd.to_datetime(fecha_inicio_consulta)
fecha_fin_consulta_ts = pd.to_datetime(fecha_fin_consulta)

# Entonces sí puedes hacer filtrado sin .apply:
filtrados = filtrados[
    (filtrados['fecha_inicio_parseada'] <= fecha_fin_consulta_ts) &
    (filtrados['fecha_fin_parseada'] >= fecha_inicio_consulta_ts)
]


In [ ]:
fecha_inicio_consulta_ts = pd.to_datetime(fecha_inicio_consulta)
fecha_fin_consulta_ts = pd.to_datetime(fecha_fin_consulta)

filtrados = filtrados[
    (filtrados['fecha_inicio_parseada'] <= fecha_fin_consulta_ts) &
    (filtrados['fecha_fin_parseada'] >= fecha_inicio_consulta_ts)
]


In [ ]:
filtrados

,ciudad,seccion,titulo,url,contenido,fecha_inicio,fecha_fin,fechas,Precios,fechas_contexto,texto_completo,fecha_inicio_parseada,fecha_fin_parseada


In [ ]:
df['fecha_inicio_parseada'].unique()

<DatetimeArray>
[                'NaT', '2025-05-15 00:00:00', '2025-05-11 00:00:00',
 '2025-06-08 00:00:00', '2025-04-26 00:00:00', '2025-05-04 00:00:00',
 '2025-05-24 00:00:00', '2025-04-27 00:00:00', '2025-05-03 00:00:00',
 '2025-04-11 00:00:00',
 ...
 '2025-08-08 00:00:00', '2025-12-05 00:00:00', '2021-07-09 00:00:00',
 '2025-10-24 00:00:00', '2025-11-10 00:00:00', '2018-02-18 00:00:00',
 '2025-04-06 00:00:00', '2025-11-07 00:00:00', '2025-10-13 00:00:00',
 '2025-12-26 00:00:00']
Length: 212, dtype: datetime64[ns]